# Gradient Checking

In [3]:
# Import necessary libraries
import numpy as np  # For numerical operations

# Import all specific function and libraries from custom modules
from testCases import *  # Import all test cases for the unit testing
from gc_utils import sigmoid, relu, vector_to_dictionary, dictionary_to_vector, gradients_to_vector

## 1) How does gradient checking works?

## 2) 1-dimensional gradient checking

In [6]:
# Import necessary library
import numpy as np  # NumPy is used for numerical operations

def forward_propagation(x, theta):
    """
    Implements linear forward propagation to compute the cost function J.
    
    J(theta) = theta * x
    
    Arguments:
    x -- a scalar or vector, representing the input(s)
    theta -- a scalar or vector, representing the parameter(s)
    
    Returns:
    J -- the computed value of the function J
    """
    
    # Compute the linear combination of inputs and parameters
    J = np.dot(theta, x)
    
    return J
    

In [7]:
# Define the input values
x, theta = 2, 4  # x is the input, theta is the parameter

# Compute the value of J using the forward_propagation function
J = forward_propagation(x, theta)

# Output the result
print(f"J = {J}")


J = 8


In [8]:
def backward_propagation(x, theta):
    """
    Computes the derivative of the cost function J with respect to the parameter theta.
    
    Arguments:
    x -- a scalar input
    theta -- a scalar parameter
    
    Returns:
    dtheta -- the computed gradient of the cost with respect to theta
    """
    
    # The derivative of J(theta) = theta * x with respect to theta is simply x
    dtheta = x
    
    return dtheta


In [9]:
# Define input values
x, theta = 2, 4  # x is the input value, theta is the parameter

# Compute the gradient of the cost function with respect to theta
dtheta = backward_propagation(x, theta)

# Print the result
print(f"dtheta = {dtheta}")


dtheta = 2


In [10]:
import numpy as np  # Import NumPy for numerical operations

def gradient_check(x, theta, epsilon=1e-7):
    """
    Performs gradient checking by comparing the analytical gradient (from backward propagation)
    to the numerical gradient (calculated using the finite difference approximation).
    
    Arguments:
    x -- a scalar input
    theta -- a scalar parameter
    epsilon -- a small value used for computing the numerical gradient approximation (default is 1e-7)
    
    Returns:
    difference -- the relative difference between the numerical gradient and the analytical gradient
    """
    
    # Compute the numerical gradient using the centered difference formula
    thetaplus = theta + epsilon    # Increment theta by epsilon
    thetaminus = theta - epsilon   # Decrement theta by epsilon
    J_plus = np.dot(thetaplus, x)  # Compute J with theta incremented
    J_minus = np.dot(thetaminus, x)  # Compute J with theta decremented
    gradapprox = (J_plus - J_minus) / (2 * epsilon)  # Numerical gradient approximation
    
    # Compute the analytical gradient using backward propagation
    grad = x  # Analytical gradient of J with respect to theta
    
    # Compute the difference between the numerical and analytical gradients
    numerator = np.linalg.norm(gradapprox - grad)  # Step 1': Compute the norm of the difference
    denominator = np.linalg.norm(gradapprox) + np.linalg.norm(grad)  # Step 2': Compute the norm of the sum
    difference = numerator / denominator  # Step 3': Relative difference
    
    # Print the result of the gradient check
    if difference < 1e-7:
        print("The gradient is correct!")
    else:
        print("The gradient is wrong!")
    
    return difference


In [11]:
# Define input values
x, theta = 2, 4  # x is the input value, theta is the parameter

# Perform gradient checking
difference = gradient_check(x, theta)

# Print the result of the gradient check
print(f"difference = {difference}")


The gradient is correct!
difference = 2.919335883291695e-10


## 3) N-dimensional gradient checking

In [13]:
import numpy as np

def forward_propagation_n(X, Y, parameters):
    """
    Implements forward propagation and computes the logistic cost function.
    
    Arguments:
    X -- Input data of shape (number of features, number of examples)
    Y -- Labels of shape (1, number of examples)
    parameters -- Dictionary containing:
        W1 -- Weight matrix of shape (5, 4)
        b1 -- Bias vector of shape (5, 1)
        W2 -- Weight matrix of shape (3, 5)
        b2 -- Bias vector of shape (3, 1)
        W3 -- Weight matrix of shape (1, 3)
        b3 -- Bias vector of shape (1, 1)
    
    Returns:
    cost -- The logistic cost function for the given examples
    cache -- A tuple containing intermediate values for backward propagation
    """
    
    # Retrieve parameters from the dictionary
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    W3 = parameters["W3"]
    b3 = parameters["b3"]

    # Forward propagation: LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID
    Z1 = np.dot(W1, X) + b1
    A1 = relu(Z1)  # Activation after first layer
    Z2 = np.dot(W2, A1) + b2
    A2 = relu(Z2)  # Activation after second layer
    Z3 = np.dot(W3, A2) + b3
    A3 = sigmoid(Z3)  # Final activation

    # Compute the cost function
    logprobs = -np.log(A3) * Y - np.log(1 - A3) * (1 - Y)
    cost = np.sum(logprobs) / X.shape[1]  # Average cost per example

    # Store intermediate values for backward propagation
    cache = (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3)
    
    return cost, cache


In [14]:
def backward_propagation_n(X, Y, cache):
    """
    Implements backward propagation to calculate the gradients of the cost function with respect to the model parameters.
    
    Arguments:
    X -- Input data of shape (input size, number of examples)
    Y -- True labels of shape (1, number of examples)
    cache -- Tuple containing intermediate values from forward_propagation_n()
    
    Returns:
    gradients -- Dictionary containing the gradients of the cost with respect to each parameter, activation, and pre-activation variable.
    """
    
    # Retrieve cached values from forward propagation
    (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache
    m = X.shape[1]  # Number of examples
    
    # Backward propagation: compute gradients for each layer
    
    # Layer 3 (Output layer): dZ3, dW3, db3
    dZ3 = A3 - Y  # Gradient of cost with respect to Z3
    dW3 = np.dot(dZ3, A2.T) / m  # Gradient of cost with respect to W3
    db3 = np.sum(dZ3, axis=1, keepdims=True) / m  # Gradient of cost with respect to b3
    
    # Layer 2: dA2, dZ2, dW2, db2
    dA2 = np.dot(W3.T, dZ3)  # Gradient of cost with respect to A2
    dZ2 = dA2 * (A2 > 0)  # Gradient of cost with respect to Z2 (ReLU derivative)
    dW2 = np.dot(dZ2, A1.T) / m  # Gradient of cost with respect to W2
    db2 = np.sum(dZ2, axis=1, keepdims=True) / m  # Gradient of cost with respect to b2
    
    # Layer 1: dA1, dZ1, dW1, db1
    dA1 = np.dot(W2.T, dZ2)  # Gradient of cost with respect to A1
    dZ1 = dA1 * (A1 > 0)  # Gradient of cost with respect to Z1 (ReLU derivative)
    dW1 = np.dot(dZ1, X.T) / m  # Gradient of cost with respect to W1
    db1 = np.sum(dZ1, axis=1, keepdims=True) / m  # Gradient of cost with respect to b1
    
    # Store gradients in a dictionary
    gradients = {
        "dZ3": dZ3, "dW3": dW3, "db3": db3,
        "dA2": dA2, "dZ2": dZ2, "dW2": dW2, "db2": db2,
        "dA1": dA1, "dZ1": dZ1, "dW1": dW1, "db1": db1
    }
    
    return gradients


In [15]:
def gradient_check_n(parameters, gradients, X, Y, epsilon=1e-7):
    """
    Verifies the correctness of the backward_propagation_n function by comparing its gradients 
    with numerically approximated gradients.
    
    Arguments:
    parameters -- dictionary containing model parameters "W1", "b1", "W2", "b2", "W3", "b3"
    gradients -- dictionary containing gradients calculated by backward_propagation_n
    X -- input data of shape (input size, number of examples)
    Y -- true label vector of shape (1, number of examples)
    epsilon -- small shift used to compute numerical gradients
    
    Returns:
    difference -- relative difference between the analytical gradients and numerical gradients
    """
    
    # Flatten parameters and gradients to vectors
    parameters_values, _ = dictionary_to_vector(parameters)
    grad_vector = gradients_to_vector(gradients)
    num_parameters = parameters_values.shape[0]

    # Initialize arrays for numerical gradient calculation
    gradapprox = np.zeros_like(parameters_values)

    # Compute the numerical gradients
    for i in range(num_parameters):
        # Compute J_plus[i]
        thetaplus = np.copy(parameters_values)
        thetaplus[i] += epsilon
        J_plus, _ = forward_propagation_n(X, Y, vector_to_dictionary(thetaplus))
        
        # Compute J_minus[i]
        thetaminus = np.copy(parameters_values)
        thetaminus[i] -= epsilon
        J_minus, _ = forward_propagation_n(X, Y, vector_to_dictionary(thetaminus))
        
        # Compute the gradient approximation
        gradapprox[i] = (J_plus - J_minus) / (2 * epsilon)
    
    # Compute the difference between analytical and numerical gradients
    numerator = np.linalg.norm(grad_vector - gradapprox)
    denominator = np.linalg.norm(grad_vector) + np.linalg.norm(gradapprox)
    difference = numerator / denominator

    # Print the result with an appropriate message
    if difference > 2e-7:
        print(f"\033[93mThere is a mistake in the backward propagation! difference = {difference}\033[0m")
    else:
        print(f"\033[92mYour backward propagation works perfectly fine! difference = {difference}\033[0m")
    
    return difference


In [16]:
# Example test case
X, Y, parameters = gradient_check_n_test_case()

# Perform forward propagation to compute the cost and cache the necessary values
cost, cache = forward_propagation_n(X, Y, parameters)

# Perform backward propagation to compute gradients with respect to the parameters
gradients = backward_propagation_n(X, Y, cache)

# Check the gradients using numerical approximation to ensure correctness
difference = gradient_check_n(parameters, gradients, X, Y)

# Print the difference between the analytical and numerical gradients
print(f"Difference between gradients: {difference}")


Your backward propagation works perfectly fine! difference = 1.1909939370580298e-07
Difference between gradients: 1.1909939370580298e-07
